# Notebook for HPV Triplicate Study

This notebook includes the steps involving the training and testing of <b> extreme gradient boosting (XGBoost) </b> machine learning models for predicting true low/intermediate-VAF <b> intrahost single nucleotide variants (iSNVs) </b> in HPV. The data files used for model training and testing in this notebook were generated using an in-house variant calling pipeline and a set of in-house scripts that processed the VCF files and parsed the variants, their replicate frequency and all the 31 features into a csv file. The notebook is indended to serve as reference for readers to understand how XGBoost model training and testing was performed in our study and more importantly, for reproducibility of the study results.

We have included the training and testing steps for the 3 scenarios described in our study: <p>
> <p> <b> 1. Machine learning with VAF filters (FM models) </b>
> <p> <b> 2. Macine learning with VCFgenie without any low-VAF filters (VM models) </b> <p>
> <p> <b> 3. Machine learning with VCFgenie with low-VAF filters (FVM models) </b> <p>

## Import libraries

In [1]:
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt
import glob
import random
from scipy import stats
from pathlib import Path
import warnings
from itertools import product
warnings.filterwarnings("ignore") # Will suppress any unnecessary warnings

# ML libraries
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, matthews_corrcoef
from sklearn.metrics import mean_squared_error, cohen_kappa_score, make_scorer
from sklearn.metrics import confusion_matrix, accuracy_score, average_precision_score, ConfusionMatrixDisplay

## Define some custom functions

In [2]:
 # Plot settings
sns.set_style('ticks')

def remove_low_coverage_samples(df_snv_w_cov, min_cov_cutoff, coverage_column):
    """
    Remove iSNVs that do not meet quality control criteria: FDP cutoff >= 100 and FAO > 0
        df_snv_w_cov: The dataframe with all SNVs and their features
        min_cov_cutoff: The minimum value for coverage
        coverage_column: The coverage column (FDP or FAO for Ion Torrent data)
    """
    df_snv_high_cov = df_snv_w_cov.copy()
    df_snv_high_cov = df_snv_high_cov.loc[df_snv_high_cov[coverage_column] >= min_cov_cutoff]
    df_snv_high_cov.reset_index(drop=True, inplace=True)
    return df_snv_high_cov
    
def create_balanced_datasets(X,y, seed_value=10, t_size=0.3, true_label=1, false_label=0):
    """
    Create balanced datasets for training. The test data will remain imbalanced.
        seed_value : The seed for random state selection
        t_size : Size of the test data. Note that this is 30% of the entire data 
                (true label + false label), not 30% of each label. You may notice that 
                the test data for the individual labels (true label or false label) are not exactly 
                30% of the entire data. 
        true_label : The numeric label for the true variants
        false_label: The numeric label for the false variants
    """
    train_samples = {} # Dictionary in which we will store the training samples
    train_index_list = [] # Will keep track of the indices of the sampled rows in the training data.
                          # Depending on the value set for alpha, the number of unique indices
                          # should equal to the size of the training data.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=t_size, random_state=seed_value)
#     print (X_train)
    X_train.reset_index(inplace=True, drop=True)
    
    # If the training data is not balanced, then undersample the over-represented class
    # and create multiple training datasets.
    num_true = y_train.count(true_label) # Number of true labels
    num_false = y_train.count(false_label) # Number of false labels
    
    # Identify the over-represented class 
    if num_true > num_false:
        over_rep_class = true_label
        over_rep_count = num_true
        under_rep_class = false_label
        under_rep_count = num_false
    elif num_false > num_true:
        over_rep_class = false_label
        over_rep_count = num_false
        under_rep_class = true_label
        under_rep_count = num_true
    else:
        # Both classes are balanced
        train_samples['1'] = {}
        train_samples['1']['X_train'] = X_train
        train_samples['1']['y_train'] = y_train
        return train_samples, X_test, y_test, train_index_list
    
    # # Commenting out this debug info statement for now. Uncomment when needed.
    # print ("\tTrain-test split resulted in imbalanced training data", flush=True)
    # print (f"\tOver-represented class = {over_rep_class}, count = {over_rep_count}", flush=True)
    # print (f"\tUnder-represented class = {under_rep_class}, count = {under_rep_count}", flush=True)
    # print ("\tWill proceed by creating balanced training data.", flush=True)
    
    alpha = 5 # A sampling weight constant that determines the number of times
            # the over-represented class will be sampled.
    num_sample_iter = int(round(over_rep_count/under_rep_count)*alpha)
    sample_size = under_rep_count
    seed_list = list(range(1,num_sample_iter+1)) # The seeds we will use for each iteration of sampling.
                                                 # This is to make the results reproducible.
    # Identify the indices of the over-represented and under-represented class
    # and get the respective data in X_train.
    over_rep_indices = list(np.where(np.array(y_train) == over_rep_class)[0])
    X_train_over_rep = X_train.iloc[over_rep_indices]
    under_rep_indices = list(np.where(np.array(y_train) == under_rep_class)[0])
    X_train_under_rep = X_train.iloc[under_rep_indices]
   
    # Perform sampling
    for seed_i in seed_list:
#         print (f"Sampling iteration {seed_i}...", flush=True, end='')
#         print (sample_size)
#         print (len(X_train_over_rep))
#         print (len(X_train_under_rep))
        X_train_sample_i_over_rep= X_train_over_rep.sample(n=sample_size, replace=False, random_state=seed_i)
        y_train_sample_i_over_rep = [over_rep_class] * sample_size
        # Consolidate the training feature data for the under represented class
        # and over-represented class into a single data frame
        X_train_sample_i = X_train_sample_i_over_rep.append(X_train_under_rep)
        
        ind_i_list = X_train_sample_i.index.tolist()
    #     print (ind_i_list)
    #     break
        if len(train_index_list) == 0:
            train_index_list = ind_i_list
        else:    
            train_index_list.extend(ind_i_list)
        
        # Turn off reset index for de-bugging
        X_train_sample_i.reset_index(inplace=True, drop=True)
        # Consolidate the training labels for the under represented
        # and over represented classes
        y_train_sample_i = y_train_sample_i_over_rep  + [under_rep_class] * sample_size
        
        # Shuffle the rows. Otherwise the Top N rows will be over-rep class and 
        # bottom N rows will be under-rep class.
        X_train_sample_i_shuffled = X_train_sample_i.sample(frac=1, random_state=seed_i)
        shuffled_indices = X_train_sample_i_shuffled.index.to_list()
        #
        y_train_sample_i_shuffled = [y_train_sample_i[ind_i] for ind_i in shuffled_indices]
        train_samples[seed_i] = {}
        train_samples[seed_i]['X_train'] = X_train_sample_i_shuffled
        train_samples[seed_i]['y_train'] = y_train_sample_i_shuffled
    return train_samples, X_test, y_test, train_index_list
 
def get_ensemble_prediction(y_scores_all_models):
    """
    For a given testing point, get the median score across all the models.
    If the median score is >= 0.5, then label 1 else label 0. 
    """
    median_scores = list(y_scores_all_models.median(axis=1))
    median_labels = list(map(lambda x: 0 if (round(x,2) < 0.5) else 1, median_scores))
    return median_labels


def get_ensemble_prediction_scores(y_scores_all_models):
    """
    Return the median scores across all the models for a given test 
    data point.
    """
    median_scores = list(y_scores_all_models.median(axis=1))
    median_scores = [round(score_i,2) for score_i in median_scores]
    return median_scores


## Performance without VCFgenie and with a lower and upper bound - <b> <font color='green'> FM models </b> </font>

### Define input and output files

In [24]:
## Create output directory
outdir = '../results/'
Path(outdir).mkdir(parents=True, exist_ok=True)


# The input vcf .csv file
snv_sbs_file = '../data/SNV_data_wo_vcf_genie.csv'

# The output file that will include the performance metrics
perf_out_file_xgb = outdir + 'performance_FM.csv'

### Define parameters and conditions

In [25]:
af_lower = [0.01,0.02,0.05,0.1]
af_upper = [0.6,0.5]
feature_cat = ['Moderate', 'Strict', 'Exhaustive']

# Below, we convert the replicate frequencies into their numerical values.
# 1 => Replicate frequency 3/3
# 0.67 => Replicate frequency 2/3
# 0.33 => Replicate frequency 1/3
true_var = [[1],[1,0.67]]
false_var = [[0.33], [0.33, 0.67]] # Skip if 0.67 is in both true_var and false_var

### Define feature categories as a dictionary

In [26]:
feat_cat_dict = {'Moderate': ['FSAF','FSAR','FSRF','FSRR','FWDB','FXX','GQ','MLLD','QUAL','REFB',
                'REVB','SAF','SAR','SRF','SRR','SSSB','STB','VARB', '5_PRIME_NUCLEOTIDE_CONTEXT', '3_PRIME_NUCLEOTIDE_CONTEXT'],
                'Strict': ['FSAF','FSAR','FSRF','FSRR','FWDB','FXX','MLLD','QUAL','REFB','REVB','SSSB','VARB', 
                           '5_PRIME_NUCLEOTIDE_CONTEXT', '3_PRIME_NUCLEOTIDE_CONTEXT'],
                'Exhaustive': ['AO','DP','FAO','FDP','FRO','FSAF','FSAR','FSRF','FSRR','FWDB',
                'FXX','GQ','HRUN','LEN','MLLD','QD','QUAL','RBI','REFB','REVB',
                'RO','SAF','SAR','SRF','SRR','SSSB','STB','STBP','VARB', '5_PRIME_NUCLEOTIDE_CONTEXT',
                '3_PRIME_NUCLEOTIDE_CONTEXT']
                }


### Generate combinations of conditions

In [27]:
param_combinations = list(product(af_lower, af_upper, feature_cat, true_var, false_var))
print (param_combinations)
print (f"Total combinations = {len(param_combinations)}")

[(0.01, 0.6, 'Moderate', [1], [0.33]), (0.01, 0.6, 'Moderate', [1], [0.33, 0.67]), (0.01, 0.6, 'Moderate', [1, 0.67], [0.33]), (0.01, 0.6, 'Moderate', [1, 0.67], [0.33, 0.67]), (0.01, 0.6, 'Strict', [1], [0.33]), (0.01, 0.6, 'Strict', [1], [0.33, 0.67]), (0.01, 0.6, 'Strict', [1, 0.67], [0.33]), (0.01, 0.6, 'Strict', [1, 0.67], [0.33, 0.67]), (0.01, 0.6, 'Exhaustive', [1], [0.33]), (0.01, 0.6, 'Exhaustive', [1], [0.33, 0.67]), (0.01, 0.6, 'Exhaustive', [1, 0.67], [0.33]), (0.01, 0.6, 'Exhaustive', [1, 0.67], [0.33, 0.67]), (0.01, 0.5, 'Moderate', [1], [0.33]), (0.01, 0.5, 'Moderate', [1], [0.33, 0.67]), (0.01, 0.5, 'Moderate', [1, 0.67], [0.33]), (0.01, 0.5, 'Moderate', [1, 0.67], [0.33, 0.67]), (0.01, 0.5, 'Strict', [1], [0.33]), (0.01, 0.5, 'Strict', [1], [0.33, 0.67]), (0.01, 0.5, 'Strict', [1, 0.67], [0.33]), (0.01, 0.5, 'Strict', [1, 0.67], [0.33, 0.67]), (0.01, 0.5, 'Exhaustive', [1], [0.33]), (0.01, 0.5, 'Exhaustive', [1], [0.33, 0.67]), (0.01, 0.5, 'Exhaustive', [1, 0.67], [0.3

<b><i> Note that for some combinations in the above, the replicate frequency 0.67 (i.e., SNV occurring in 2/3 replicates) is used both for true variants and false variants. Example of such a combination is (0.1, 0.5, 'Exhaustive', [1, 0.67], [0.33, 0.67]). We will exclude such combinations in our analysis, which will result in 72 combinations in total.

### Run Xtreme Gradient Boosting iteratively on these conditions

In [28]:
df_snv_sbs = pd.read_csv(snv_sbs_file)
df_perf_out_xgb = pd.DataFrame()

# Use FDP coverage cutoff = 100 and remove low coverage samples and variants
coverage_cutoff = 100
df_snv_sbs = remove_low_coverage_samples(df_snv_sbs, coverage_cutoff, 'FDP')

# Remove variants with FAO = 0
fao_cutoff = 1
df_snv_sbs = remove_low_coverage_samples(df_snv_sbs, fao_cutoff, 'FAO')

# To eliminate the performance bias due to sampling, we will use a set of seeds to perform
# random sampling. Using just a single seed might bias the testing data sampled
# and a very good performance on just one testing dataset is highly indicative of
# performance bias.
seed_list = [10,20,3247,24,4501,9879,878,76,187,299]

for seed_i in seed_list:
    print (f"Running predictions for all parameter/feature combinations with seed {seed_i}...", end="", flush=True)
    df_perf_seed_i = pd.DataFrame()
    for comb_i in param_combinations:
        df_xgb_all_data = df_snv_sbs.copy()
        af_lower_i, af_upper_i, feat_cat_i, true_var_i, false_var_i = comb_i
        
        # Commenting these print statements to reduce the amount of output text in the notebook
        # Uncomment if you desire to see the combinations
        #print (f"Evaluating with parameters: af_lower_i={af_lower_i}, af_upper_i={af_upper_i}, feat_cat_i={feat_cat_i}, true_var_i={true_var_i}, false_var_i={false_var_i}")
    
        # If the 2/3 variant is included in both the true variant and false variant
        # then exclude that combination
        if 0.67 in true_var_i and 0.67 in false_var_i:
            continue
        feature_cols = feat_cat_dict[feat_cat_i]
        df_xgb_all_data = df_xgb_all_data.loc[(df_xgb_all_data['AF'] > af_lower_i) & 
                                             (df_xgb_all_data['AF'] < af_upper_i)]
        if len(true_var_i) == 1 and len(false_var_i) == 1:
            df_xgb_all_data = df_xgb_all_data.loc[(df_xgb_all_data['PERCENT_OVERLAP'] == true_var_i[0]) |
                                                (df_xgb_all_data['PERCENT_OVERLAP'] == false_var_i[0])]

        df_xgb_all_data.reset_index(drop=True, inplace=True)
        if len(true_var_i) == 1:
            overlap_cat = list(df_xgb_all_data['PERCENT_OVERLAP'].apply(lambda x: 1 if (x == true_var_i[0]) else 0 ))
        else:
            overlap_cat = list(df_xgb_all_data['PERCENT_OVERLAP'].apply(lambda x: 1 if (x == true_var_i[0] or x == true_var_i[1]) else 0 ))

        df_xgb_all_data['OVERLAP_CATEGORY'] = overlap_cat
        target_cols = 'OVERLAP_CATEGORY'

        # Define X and y 
        X = df_xgb_all_data[feature_cols]
        y = df_xgb_all_data[target_cols].tolist()

        # Create balanced datasets
        train_sample, X_test, y_test, train_index_list = create_balanced_datasets(X,y, seed_i)

        # Train model
        num_jobs = 6
        n_trees = 100
        mdepth = 10
        xgb_model = XGBClassifier(use_label_encoder=False, 
                              booster='gbtree', # boosting algorithm to use, default gbtree, othera: gblinear, dart
                              n_estimators=n_trees, # number of trees, default = 100
                              eta=0.2, # this is learning rate, default = 0.3
                              max_depth=mdepth, # maximum depth of the tree, default = 6
                              gamma = 1, # used for pruning, if gain < gamma the branch will be pruned, default = 0
                              reg_lambda = 1, # regularization parameter, defautl = 1
                              eval_metric = 'logloss'
                             )
        fpr_tpr = [] # Store all the fpr, tpr values for each model 
        auc_scores = []
        df_y_pred_all_models = pd.DataFrame()
        df_y_scores_all_models = pd.DataFrame()
        df_all_cv_scores = pd.DataFrame()
        df_feature_importances_all_models = pd.DataFrame()
        for model_i in list(train_sample.keys()):
            X_train_model_i = train_sample[model_i]['X_train']
            y_train_model_i = train_sample[model_i]['y_train']
    #         print (f"Running CV and predictions for model {model_i}...", end='', flush=True)
            X_train_model_i = X_train_model_i.apply(pd.to_numeric)
            xgb_model.fit(X_train_model_i,y_train_model_i)

            # Predict using model_i on the test data
            y_pred = xgb_model.predict(X_test)
            df_y_pred_all_models[f"Model_{model_i}"] = y_pred
            y_test_pred_scores = xgb_model.predict_proba(X_test)[:, 1]
            df_y_scores_all_models[f"Model_{model_i}"] = y_test_pred_scores
            false_positive, true_positive, _ = roc_curve(y_test, y_test_pred_scores)
            fpr_tpr.append([false_positive,true_positive])
            auc_test = roc_auc_score(y_test, y_pred)
            auc_scores.append(auc_test)
        # Get the consensus predictions from all models
        y_pred_consensus = get_ensemble_prediction(df_y_scores_all_models)

        # Calculate metrics on the consensus prediction
        auc_test = roc_auc_score(y_test, y_pred_consensus)

        # Calculate the F1 score
        F1_score_test = round(f1_score(y_test, y_pred_consensus), 3)

        # Calculate the mean-squared error
        mse_test = round(mean_squared_error(y_test, y_pred_consensus), 3)

        # Calculate the accuracy
        accuracy_test = round(accuracy_score(y_test, y_pred_consensus), 3)

        # Calculate the MCC
        mcc_test = round(matthews_corrcoef(y_test, y_pred_consensus), 3)

        # Print the performance metrics on test data
        # Uncomment to see the metrics for each combination
        #print ("AUC =", round(auc_test, 3), ", F1 score =", F1_score_test, ", Mean-squared error =", mse_test, ", Accuracy =", accuracy_test, ", Matthews correlation coefficient =", mcc_test)

        # Get the confusion matrix
        cm = confusion_matrix(y_test, y_pred_consensus)

        # Get the indices of false positives
        false_positive_ind = list(set(np.where(np.array(y_test) == 0)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 1)[0])))
        # Get the indices of true positives
        true_positive_ind = list(set(np.where(np.array(y_test) == 1)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 1)[0])))
        # Get the indices of true negatives
        true_negative_ind = list(set(np.where(np.array(y_test) == 0)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 0)[0])))
        # Get the indices of false negatives
        false_negative_ind = list(set(np.where(np.array(y_test) == 1)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 0)[0])))

        fp = len(false_positive_ind)
        tp = len(true_positive_ind)
        tn = len(true_negative_ind)
        fn = len(false_negative_ind)

        cols = ['Seed', 'True_Var_Def', 'False_Var_Def', 'VAF_Lower_Limit', 'VAF_Upper_Limit',  'Feature_Category',
                'Num_True_Var_Testing', 'Num_False_Var_Testing', 'TP', 'FP', 'TN', 'FN', 'AUC', 'MCC', 
                'F1_score', 'Accuracy', 'MSE']

        # Add an additional space before the true var def and false var def. Otherwise, excel formats it as date.
        df_tmp = pd.DataFrame(data=[[seed_i, true_var_i, false_var_i,af_lower_i,af_upper_i,feat_cat_i, 
                                     tp+fn, tn+fp, tp, fp, tn, fn, auc_test, mcc_test, F1_score_test,
                                     accuracy_test, mse_test]], columns=cols)
        df_perf_seed_i = df_perf_seed_i.append(df_tmp)
        #break
    df_perf_out_xgb = df_perf_out_xgb.append(df_perf_seed_i)
    print ("Done!")
df_perf_out_xgb.to_csv(perf_out_file_xgb, index=False)
df_perf_out_xgb 

Running predictions for all parameter/feature combinations with seed 10...Done!
Running predictions for all parameter/feature combinations with seed 20...Done!
Running predictions for all parameter/feature combinations with seed 3247...Done!
Running predictions for all parameter/feature combinations with seed 24...Done!
Running predictions for all parameter/feature combinations with seed 4501...Done!
Running predictions for all parameter/feature combinations with seed 9879...Done!
Running predictions for all parameter/feature combinations with seed 878...Done!
Running predictions for all parameter/feature combinations with seed 76...Done!
Running predictions for all parameter/feature combinations with seed 187...Done!
Running predictions for all parameter/feature combinations with seed 299...Done!


,Seed,True_Var_Def,False_Var_Def,VAF_Lower_Limit,VAF_Upper_Limit,Feature_Category,Num_True_Var_Testing,Num_False_Var_Testing,TP,FP,TN,FN,AUC,MCC,F1_score,Accuracy,MSE
0,10,[1],[0.33],0.01,0.6,Moderate,159,1055,123,230,825,36,0.777788,0.413,0.480,0.781,0.219
0,10,[1],"[0.33, 0.67]",0.01,0.6,Moderate,134,1243,104,253,990,30,0.786290,0.387,0.424,0.794,0.206
0,10,"[1, 0.67]",[0.33],0.01,0.6,Moderate,287,1090,187,310,780,100,0.683582,0.311,0.477,0.702,0.298
0,10,[1],[0.33],0.01,0.6,Strict,159,1055,126,240,815,33,0.782482,0.415,0.480,0.775,0.225
0,10,[1],"[0.33, 0.67]",0.01,0.6,Strict,134,1243,104,267,976,30,0.780658,0.375,0.412,0.784,0.216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,299,[1],"[0.33, 0.67]",0.10,0.5,Strict,41,88,26,34,54,15,0.623891,0.231,0.515,0.620,0.380
0,299,"[1, 0.67]",[0.33],0.10,0.5,Strict,56,73,28,33,40,28,0.523973,0.048,0.479,0.527,0.473
0,299,[1],[0.33],0.10,0.5,Exhaustive,38,72,26,28,44,12,0.647661,0.281,0.565,0.636,0.364
0,299,[1],"[0.33, 0.67]",0.10,0.5,Exhaustive,41,88,23,25,63,18,0.638442,0.267,0.517,0.667,0.333


## Performance with results from VCFgenie, but without a lower bound on VAF - <b> <font color='orange'> VM models </font> </b>

### Define input and output files

In [13]:
# The input vcf .csv file
snv_sbs_file = '../data/SNV_data_with_vcf_genie.csv'
perf_out_file_xgb = outdir + 'performance_VM.csv'

### Define parameters and conditions

In [14]:
af_upper = [0.6,0.5]
feature_cat = ['Moderate', 'Strict', 'Exhaustive']
true_var = [[1],[1,0.67]]
false_var = [[0.33], [0.33, 0.67]] # Skip if 0.67 is in both true_var and false_var


### Define features categories as a dictionary

In [15]:
feat_cat_dict = {'Moderate': ['FSAF','FSAR','FSRF','FSRR','FWDB','FXX','GQ','MLLD','QUAL','REFB',
                'REVB','SAF','SAR','SRF','SRR','SSSB','STB','VARB', '5_PRIME_NUCLEOTIDE_CONTEXT', '3_PRIME_NUCLEOTIDE_CONTEXT'],
                'Strict': ['FSAF','FSAR','FSRF','FSRR','FWDB','FXX','MLLD','QUAL','REFB','REVB','SSSB','VARB', 
                           '5_PRIME_NUCLEOTIDE_CONTEXT', '3_PRIME_NUCLEOTIDE_CONTEXT'],
                'Exhaustive': ['AO','DP','FAO','FDP','FRO','FSAF','FSAR','FSRF','FSRR','FWDB',
                'FXX','GQ','HRUN','LEN','MLLD','QD','QUAL','RBI','REFB','REVB',
                'RO','SAF','SAR','SRF','SRR','SSSB','STB','STBP','VARB', '5_PRIME_NUCLEOTIDE_CONTEXT',
                '3_PRIME_NUCLEOTIDE_CONTEXT']
                }


### Generate combinations of conditions

In [16]:
param_combinations = list(product(af_upper, feature_cat, true_var, false_var))
print (param_combinations)
print (f"Total combinations = {len(param_combinations)}")

[(0.6, 'Moderate', [1], [0.33]), (0.6, 'Moderate', [1], [0.33, 0.67]), (0.6, 'Moderate', [1, 0.67], [0.33]), (0.6, 'Moderate', [1, 0.67], [0.33, 0.67]), (0.6, 'Strict', [1], [0.33]), (0.6, 'Strict', [1], [0.33, 0.67]), (0.6, 'Strict', [1, 0.67], [0.33]), (0.6, 'Strict', [1, 0.67], [0.33, 0.67]), (0.6, 'Exhaustive', [1], [0.33]), (0.6, 'Exhaustive', [1], [0.33, 0.67]), (0.6, 'Exhaustive', [1, 0.67], [0.33]), (0.6, 'Exhaustive', [1, 0.67], [0.33, 0.67]), (0.5, 'Moderate', [1], [0.33]), (0.5, 'Moderate', [1], [0.33, 0.67]), (0.5, 'Moderate', [1, 0.67], [0.33]), (0.5, 'Moderate', [1, 0.67], [0.33, 0.67]), (0.5, 'Strict', [1], [0.33]), (0.5, 'Strict', [1], [0.33, 0.67]), (0.5, 'Strict', [1, 0.67], [0.33]), (0.5, 'Strict', [1, 0.67], [0.33, 0.67]), (0.5, 'Exhaustive', [1], [0.33]), (0.5, 'Exhaustive', [1], [0.33, 0.67]), (0.5, 'Exhaustive', [1, 0.67], [0.33]), (0.5, 'Exhaustive', [1, 0.67], [0.33, 0.67])]
Total combinations = 24


### Run Xtreme Gradient Boosting iteratively on these conditions

In [17]:
df_snv_sbs = pd.read_csv(snv_sbs_file)
df_perf_out_xgb = pd.DataFrame()

# Use FDP coverage cutoff = 100 and remove low coverage samples and variants
coverage_cutoff = 100
print ("Removing low FDP variants...", end='', flush=True)
df_snv_sbs = remove_low_coverage_samples(df_snv_sbs, coverage_cutoff, 'FDP')
print ("Done!")

# Remove variants with FAO = 0
fao_cutoff = 1
print ("Removing low FAO variants...", end='', flush=True)
df_snv_sbs = remove_low_coverage_samples(df_snv_sbs, fao_cutoff, 'FAO')
print ("Done!")

seed_list = [10,20,3247,24,4501,9879,878,76,187,299]
for seed_i in seed_list:
    print (f"Running predictions for all parameter/feature combinations with seed {seed_i}...", end="", flush=True)
    df_perf_seed_i = pd.DataFrame()
    for comb_i in param_combinations:
        df_xgb_all_data = df_snv_sbs.copy()
        af_upper_i, feat_cat_i, true_var_i, false_var_i = comb_i
        #print (f"Evaluating with parameters: af_upper_i={af_upper_i}, feat_cat_i={feat_cat_i}, true_var_i={true_var_i}, false_var_i={false_var_i}")
    #     print(af_lower_i, af_upper_i, feat_cat_i, true_var_i, false_var_i, exclude_lin_pos_i)

        # If the 2/3 variant is included in both the true variant and false variant
        # then exclude that combination
        if 0.67 in true_var_i and 0.67 in false_var_i:
            continue
        feature_cols = feat_cat_dict[feat_cat_i]
        df_xgb_all_data = df_xgb_all_data.loc[(df_xgb_all_data['AF'] < af_upper_i)]

        if len(true_var_i) == 1 and len(false_var_i) == 1:
            df_xgb_all_data = df_xgb_all_data.loc[(df_xgb_all_data['PERCENT_OVERLAP'] == true_var_i[0]) |
                                                (df_xgb_all_data['PERCENT_OVERLAP'] == false_var_i[0])]

        df_xgb_all_data.reset_index(drop=True, inplace=True)
        if len(true_var_i) == 1:
            overlap_cat = list(df_xgb_all_data['PERCENT_OVERLAP'].apply(lambda x: 1 if (x == true_var_i[0]) else 0 ))
        else:
            overlap_cat = list(df_xgb_all_data['PERCENT_OVERLAP'].apply(lambda x: 1 if (x == true_var_i[0] or x == true_var_i[1]) else 0 ))

        df_xgb_all_data['OVERLAP_CATEGORY'] = overlap_cat
        target_cols = 'OVERLAP_CATEGORY'

        # Define X and y 
        X = df_xgb_all_data[feature_cols]
        y = df_xgb_all_data[target_cols].tolist()

        # Create balanced datasets
        train_sample, X_test, y_test, train_index_list = create_balanced_datasets(X,y,seed_i)

        # Train model
        num_jobs = 6
        n_trees = 100
        mdepth = 10
        xgb_model = XGBClassifier(use_label_encoder=False, 
                              booster='gbtree', # boosting algorithm to use, default gbtree, othera: gblinear, dart
                              n_estimators=n_trees, # number of trees, default = 100
                              eta=0.2, # this is learning rate, default = 0.3
                              max_depth=mdepth, # maximum depth of the tree, default = 6
                              gamma = 1, # used for pruning, if gain < gamma the branch will be pruned, default = 0
                              reg_lambda = 1, # regularization parameter, defautl = 1
                              eval_metric = 'logloss'
                             )
        fpr_tpr = [] # Store all the fpr, tpr values for each model 
        auc_scores = []
        df_y_pred_all_models = pd.DataFrame()
        df_y_scores_all_models = pd.DataFrame()
        df_all_cv_scores = pd.DataFrame()
        df_feature_importances_all_models = pd.DataFrame()
        for model_i in list(train_sample.keys()):
            X_train_model_i = train_sample[model_i]['X_train']
            y_train_model_i = train_sample[model_i]['y_train']
    #         print (f"Running CV and predictions for model {model_i}...", end='', flush=True)
            X_train_model_i = X_train_model_i.apply(pd.to_numeric)
            xgb_model.fit(X_train_model_i,y_train_model_i)

            # Predict using model_i on the test data
            y_pred = xgb_model.predict(X_test)
            df_y_pred_all_models[f"Model_{model_i}"] = y_pred
            y_test_pred_scores = xgb_model.predict_proba(X_test)[:, 1]
            df_y_scores_all_models[f"Model_{model_i}"] = y_test_pred_scores
            false_positive, true_positive, _ = roc_curve(y_test, y_test_pred_scores)
            fpr_tpr.append([false_positive,true_positive])
            auc_test = roc_auc_score(y_test, y_pred)
            auc_scores.append(auc_test)
        # Get the consensus predictions from all models
        y_pred_consensus = get_ensemble_prediction(df_y_scores_all_models)

        # Calculate metrics on the consensus prediction
        auc_test = roc_auc_score(y_test, y_pred_consensus)

        # Calculate the F1 score
        F1_score_test = round(f1_score(y_test, y_pred_consensus), 3)

        # Calculate the mean-squared error
        mse_test = round(mean_squared_error(y_test, y_pred_consensus), 3)

        # Calculate the accuracy
        accuracy_test = round(accuracy_score(y_test, y_pred_consensus), 3)

        # Calculate the MCC
        mcc_test = round(matthews_corrcoef(y_test, y_pred_consensus), 3)

        # Print the performance metrics on test data
        #print ("AUC =", round(auc_test, 3), ", F1 score =", F1_score_test, ", Mean-squared error =", mse_test, ", Accuracy =", accuracy_test, ", Matthews correlation coefficient =", mcc_test)
        
        # Get the confusion matrix
        cm = confusion_matrix(y_test, y_pred_consensus)

        # Get the indices of false positives
        false_positive_ind = list(set(np.where(np.array(y_test) == 0)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 1)[0])))
        # Get the indices of true positives
        true_positive_ind = list(set(np.where(np.array(y_test) == 1)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 1)[0])))
        # Get the indices of true negatives
        true_negative_ind = list(set(np.where(np.array(y_test) == 0)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 0)[0])))
        # Get the indices of false negatives
        false_negative_ind = list(set(np.where(np.array(y_test) == 1)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 0)[0])))

        fp = len(false_positive_ind)
        tp = len(true_positive_ind)
        tn = len(true_negative_ind)
        fn = len(false_negative_ind)

        cols = ['Seed', 'True_Var_Def', 'False_Var_Def', 'VAF_Upper_Limit', 'Feature_Category', 
                'Num_True_Var_Testing', 'Num_False_Var_Testing', 'TP', 'FP', 'TN', 'FN', 'AUC', 'MCC', 
                'F1_score', 'Accuracy', 'MSE']

        # Add an additional space before the true var def and false var def. Otherwise, excel formats it as date.
        df_tmp = pd.DataFrame(data=[[seed_i, true_var_i, false_var_i,af_upper_i,feat_cat_i, tp+fn,
                                     tn+fp, tp, fp, tn, fn, auc_test, mcc_test, F1_score_test,
                                     accuracy_test, mse_test]], columns=cols)
        df_perf_seed_i = df_perf_seed_i.append(df_tmp)

        #break
    df_perf_out_xgb =  df_perf_out_xgb.append(df_perf_seed_i)
    print ("Done!")
df_perf_out_xgb.to_csv(perf_out_file_xgb, index=False)

Removing low FDP variants...Done!
Removing low FAO variants...Done!
***Running predictions for all parameter/feature combinations with seed 10***Done!
***Running predictions for all parameter/feature combinations with seed 20***Done!
***Running predictions for all parameter/feature combinations with seed 3247***Done!
***Running predictions for all parameter/feature combinations with seed 24***Done!
***Running predictions for all parameter/feature combinations with seed 4501***Done!
***Running predictions for all parameter/feature combinations with seed 9879***Done!
***Running predictions for all parameter/feature combinations with seed 878***Done!
***Running predictions for all parameter/feature combinations with seed 76***Done!
***Running predictions for all parameter/feature combinations with seed 187***Done!
***Running predictions for all parameter/feature combinations with seed 299***Done!


## Performance with results from VCFgenie and with a lower VAF and upper VAF cutoff - <b> <font color='royalblue'> FVM models

### Define input and output files

In [19]:
# The input vcf .csv file

snv_sbs_file = '../data/SNV_data_with_vcf_genie.csv'
perf_out_file_xgb = outdir + 'performance_FVM.csv'

### Define parameter and conditions

In [20]:
af_lower = [0.01,0.02,0.05,0.1]
af_upper = [0.6,0.5]
feature_cat = ['Moderate', 'Strict', 'Exhaustive']
true_var = [[1],[1,0.67]]
false_var = [[0.33], [0.33, 0.67]] # Skip if 0.67 is in both true_var and false_var

### Define feature categories as a dictionary

In [21]:
feat_cat_dict = {'Moderate': ['FSAF','FSAR','FSRF','FSRR','FWDB','FXX','GQ','MLLD','QUAL','REFB',
                'REVB','SAF','SAR','SRF','SRR','SSSB','STB','VARB', '5_PRIME_NUCLEOTIDE_CONTEXT', '3_PRIME_NUCLEOTIDE_CONTEXT'],
                'Strict': ['FSAF','FSAR','FSRF','FSRR','FWDB','FXX','MLLD','QUAL','REFB','REVB','SSSB','VARB', 
                           '5_PRIME_NUCLEOTIDE_CONTEXT', '3_PRIME_NUCLEOTIDE_CONTEXT'],
                'Exhaustive': ['AO','DP','FAO','FDP','FRO','FSAF','FSAR','FSRF','FSRR','FWDB',
                'FXX','GQ','HRUN','LEN','MLLD','QD','QUAL','RBI','REFB','REVB',
                'RO','SAF','SAR','SRF','SRR','SSSB','STB','STBP','VARB', '5_PRIME_NUCLEOTIDE_CONTEXT',
                '3_PRIME_NUCLEOTIDE_CONTEXT']
                }


### Generate combinations of conditions

In [22]:
param_combinations = list(product(af_lower, af_upper, feature_cat, true_var, false_var))
print (param_combinations)
print (f"Total combinations = {len(param_combinations)}")

[(0.01, 0.6, 'Moderate', [1], [0.33]), (0.01, 0.6, 'Moderate', [1], [0.33, 0.67]), (0.01, 0.6, 'Moderate', [1, 0.67], [0.33]), (0.01, 0.6, 'Moderate', [1, 0.67], [0.33, 0.67]), (0.01, 0.6, 'Strict', [1], [0.33]), (0.01, 0.6, 'Strict', [1], [0.33, 0.67]), (0.01, 0.6, 'Strict', [1, 0.67], [0.33]), (0.01, 0.6, 'Strict', [1, 0.67], [0.33, 0.67]), (0.01, 0.6, 'Exhaustive', [1], [0.33]), (0.01, 0.6, 'Exhaustive', [1], [0.33, 0.67]), (0.01, 0.6, 'Exhaustive', [1, 0.67], [0.33]), (0.01, 0.6, 'Exhaustive', [1, 0.67], [0.33, 0.67]), (0.01, 0.5, 'Moderate', [1], [0.33]), (0.01, 0.5, 'Moderate', [1], [0.33, 0.67]), (0.01, 0.5, 'Moderate', [1, 0.67], [0.33]), (0.01, 0.5, 'Moderate', [1, 0.67], [0.33, 0.67]), (0.01, 0.5, 'Strict', [1], [0.33]), (0.01, 0.5, 'Strict', [1], [0.33, 0.67]), (0.01, 0.5, 'Strict', [1, 0.67], [0.33]), (0.01, 0.5, 'Strict', [1, 0.67], [0.33, 0.67]), (0.01, 0.5, 'Exhaustive', [1], [0.33]), (0.01, 0.5, 'Exhaustive', [1], [0.33, 0.67]), (0.01, 0.5, 'Exhaustive', [1, 0.67], [0.3

### Run Xtreme Gradient Boosting iteratively in these conditions

In [23]:
df_snv_sbs = pd.read_csv(snv_sbs_file)
df_perf_out_xgb = pd.DataFrame()

# Use FDP coverage cutoff = 100 and remove low coverage samples and variants
coverage_cutoff = 100
print ("Removing low FDP variants...", end='', flush=True)
df_snv_sbs = remove_low_coverage_samples(df_snv_sbs, coverage_cutoff, 'FDP')
print ("Done!")

# Remove variants with FAO = 0
fao_cutoff = 1
print ("Removing low FAO variants...", end='', flush=True)
df_snv_sbs = remove_low_coverage_samples(df_snv_sbs, fao_cutoff, 'FAO')
print ("Done!")

seed_list = [10,20,3247,24,4501,9879,878,76,187,299]
for seed_i in seed_list:
    print (f"Running predictions for all parameter/feature combinations with seed {seed_i}...", end="", flush=True)
    df_perf_seed_i = pd.DataFrame()
    for comb_i in param_combinations:
        df_xgb_all_data = df_snv_sbs.copy()
        af_lower_i, af_upper_i, feat_cat_i, true_var_i, false_var_i = comb_i
        #print (f"Evaluating with parameters:\n af_lower_i={af_lower_i}, af_upper_i={af_upper_i}, feat_cat_i={feat_cat_i}, true_var_i={true_var_i}, false_var_i={false_var_i}")
    #     print(af_lower_i, af_upper_i, feat_cat_i, true_var_i, false_var_i, exclude_lin_pos_i)

        # If the 2/3 variant is included in both the true variant and false variant
        # then exclude that combination
        if 0.67 in true_var_i and 0.67 in false_var_i:
            continue
        feature_cols = feat_cat_dict[feat_cat_i]
        df_xgb_all_data = df_xgb_all_data.loc[(df_xgb_all_data['AF'] > af_lower_i) & 
                                                (df_xgb_all_data['AF'] < af_upper_i)]
        if len(true_var_i) == 1 and len(false_var_i) == 1:
            df_xgb_all_data = df_xgb_all_data.loc[(df_xgb_all_data['PERCENT_OVERLAP'] == true_var_i[0]) |
                                                (df_xgb_all_data['PERCENT_OVERLAP'] == false_var_i[0])]

        df_xgb_all_data.reset_index(drop=True, inplace=True)
        if len(true_var_i) == 1:
            overlap_cat = list(df_xgb_all_data['PERCENT_OVERLAP'].apply(lambda x: 1 if (x == true_var_i[0]) else 0 ))
        else:
            overlap_cat = list(df_xgb_all_data['PERCENT_OVERLAP'].apply(lambda x: 1 if (x == true_var_i[0] or x == true_var_i[1]) else 0 ))

        df_xgb_all_data['OVERLAP_CATEGORY'] = overlap_cat
        target_cols = 'OVERLAP_CATEGORY'

        # Define X and y 
        X = df_xgb_all_data[feature_cols]
        y = df_xgb_all_data[target_cols].tolist()

        # Create balanced datasets
        train_sample, X_test, y_test, train_index_list = create_balanced_datasets(X,y,seed_i)

        # Train model
        num_jobs = 6
        n_trees = 100
        mdepth = 10
        xgb_model = XGBClassifier(use_label_encoder=False, 
                              booster='gbtree', # boosting algorithm to use, default gbtree, othera: gblinear, dart
                              n_estimators=n_trees, # number of trees, default = 100
                              eta=0.2, # this is learning rate, default = 0.3
                              max_depth=mdepth, # maximum depth of the tree, default = 6
                              gamma = 1, # used for pruning, if gain < gamma the branch will be pruned, default = 0
                              reg_lambda = 1, # regularization parameter, defautl = 1
                              eval_metric = 'logloss'
                             )
        fpr_tpr = [] # Store all the fpr, tpr values for each model 
        auc_scores = []
        df_y_pred_all_models = pd.DataFrame()
        df_y_scores_all_models = pd.DataFrame()
        df_all_cv_scores = pd.DataFrame()
        df_feature_importances_all_models = pd.DataFrame()
        for model_i in list(train_sample.keys()):
            X_train_model_i = train_sample[model_i]['X_train']
            y_train_model_i = train_sample[model_i]['y_train']
    #         print (f"Running CV and predictions for model {model_i}...", end='', flush=True)
            X_train_model_i = X_train_model_i.apply(pd.to_numeric)
            xgb_model.fit(X_train_model_i,y_train_model_i)

            # Predict using model_i on the test data
            y_pred = xgb_model.predict(X_test)
            df_y_pred_all_models[f"Model_{model_i}"] = y_pred
            y_test_pred_scores = xgb_model.predict_proba(X_test)[:, 1]
            df_y_scores_all_models[f"Model_{model_i}"] = y_test_pred_scores
            false_positive, true_positive, _ = roc_curve(y_test, y_test_pred_scores)
            fpr_tpr.append([false_positive,true_positive])
            auc_test = roc_auc_score(y_test, y_pred)
            auc_scores.append(auc_test)
        # Get the consensus predictions from all models
        y_pred_consensus = get_ensemble_prediction(df_y_scores_all_models)

        # Calculate metrics on the consensus prediction
        auc_test = roc_auc_score(y_test, y_pred_consensus)

        # Calculate the F1 score
        F1_score_test = round(f1_score(y_test, y_pred_consensus), 3)

        # Calculate the mean-squared error
        mse_test = round(mean_squared_error(y_test, y_pred_consensus), 3)

        # Calculate the accuracy
        accuracy_test = round(accuracy_score(y_test, y_pred_consensus), 3)

        # Calculate the MCC
        mcc_test = round(matthews_corrcoef(y_test, y_pred_consensus), 3)

        # Print the performance metrics on test data
        # print ("\tAUC =", round(auc_test, 3), "\n\tF1 score =", F1_score_test, "\n\tMean-squared error =", mse_test, "\n\tAccuracy =", accuracy_test, "\n\tMatthews correlation coefficient =", mcc_test)

        # Get the confusion matrix
        cm = confusion_matrix(y_test, y_pred_consensus)

        # Get the indices of false positives
        false_positive_ind = list(set(np.where(np.array(y_test) == 0)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 1)[0])))
        # Get the indices of true positives
        true_positive_ind = list(set(np.where(np.array(y_test) == 1)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 1)[0])))
        # Get the indices of true negatives
        true_negative_ind = list(set(np.where(np.array(y_test) == 0)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 0)[0])))
        # Get the indices of false negatives
        false_negative_ind = list(set(np.where(np.array(y_test) == 1)[0]).intersection(set(np.where(np.array(y_pred_consensus) == 0)[0])))

        fp = len(false_positive_ind)
        tp = len(true_positive_ind)
        tn = len(true_negative_ind)
        fn = len(false_negative_ind)

        cols = ['Seed','True_Var_Def', 'False_Var_Def', 'VAF_Lower_Limit', 'VAF_Upper_Limit', 'Feature_Category', 
                'Num_True_Var_Testing', 'Num_False_Var_Testing', 'TP', 'FP', 'TN', 'FN', 'AUC', 'MCC', 
                'F1_score', 'Accuracy', 'MSE']

        # Add an additional space before the true var def and false var def. Otherwise, excel formats it as date.
        df_tmp = pd.DataFrame(data=[[seed_i, true_var_i, false_var_i,af_lower_i,af_upper_i,feat_cat_i, 
                                     tp+fn, tn+fp, tp, fp, tn, fn, auc_test, mcc_test, F1_score_test,
                                     accuracy_test, mse_test]], columns=cols)
        df_perf_seed_i = df_perf_seed_i.append(df_tmp)
    df_perf_out_xgb = df_perf_out_xgb.append(df_perf_seed_i)
    print ("Done!")
df_perf_out_xgb.to_csv(perf_out_file_xgb, index=False)

Removing low FDP variants...Done!
Removing low FAO variants...Done!
Running predictions for all parameter/feature combinations with seed 10...Done!
Running predictions for all parameter/feature combinations with seed 20...Done!
Running predictions for all parameter/feature combinations with seed 3247...Done!
Running predictions for all parameter/feature combinations with seed 24...Done!
Running predictions for all parameter/feature combinations with seed 4501...Done!
Running predictions for all parameter/feature combinations with seed 9879...Done!
Running predictions for all parameter/feature combinations with seed 878...Done!
Running predictions for all parameter/feature combinations with seed 76...Done!
Running predictions for all parameter/feature combinations with seed 187...Done!
Running predictions for all parameter/feature combinations with seed 299...Done!
